# 🔴 Avancé | ⏱ 60 min | 🔑 Concepts : Singleton, Factory, Strategy, Observer, Decorator pattern

# Design Patterns en Python

## 🎯 Objectifs

- Comprendre ce qu'est un design pattern et pourquoi les utiliser
- Maîtriser les patterns essentiels : Singleton, Factory, Strategy, Observer
- Distinguer le Decorator pattern des @decorators Python
- Adapter les patterns classiques à l'approche pythonique
- Savoir quand utiliser (ou ne pas utiliser) un pattern

## 📚 Prérequis

- POO avancée en Python
- Héritage et polymorphisme
- Méthodes spéciales (`__new__`, `__call__`)
- Compréhension des closures et fonctions de première classe

## 1. Qu'est-ce qu'un Design Pattern ?

Un **design pattern** (patron de conception) est une solution réutilisable à un problème courant dans la conception logicielle.

### Origines

- Popularisés par le "Gang of Four" (GoF) dans leur livre de 1994
- 23 patterns classiques organisés en 3 catégories

### Catégories

1. **Patterns de création** : comment créer des objets
   - Singleton, Factory, Builder, Prototype

2. **Patterns de structure** : comment organiser les classes
   - Adapter, Decorator, Facade, Proxy

3. **Patterns de comportement** : comment les objets interagissent
   - Strategy, Observer, Command, Iterator

### Pourquoi les utiliser ?

- **Vocabulaire commun** : facilite la communication entre développeurs
- **Solutions éprouvées** : évite de réinventer la roue
- **Code maintenable** : structure claire et compréhensible
- **Flexibilité** : facilite les évolutions futures

### Attention !

- Ne pas sur-utiliser : KISS (Keep It Simple, Stupid)
- Python a des idiomes qui remplacent certains patterns
- Adapter les patterns au style pythonique

## 2. Singleton : Une Seule Instance

Le pattern **Singleton** garantit qu'une classe n'a qu'une seule instance et fournit un point d'accès global.

### Cas d'usage
- Configuration globale
- Logger
- Connexion à une base de données
- Cache partagé

### Implémentations en Python

In [ ]:
# Méthode 1 : Utiliser __new__
class SingletonNew:
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            print("Création de l'instance Singleton")
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, valeur=None):
        # Attention : __init__ est appelé à chaque fois !
        if not hasattr(self, 'initialized'):
            self.valeur = valeur
            self.initialized = True
            print(f"Initialisation avec valeur={valeur}")

# Test
print("Création de s1")
s1 = SingletonNew("première valeur")
print(f"s1.valeur : {s1.valeur}")
print(f"id(s1) : {id(s1)}")

print("\nCréation de s2")
s2 = SingletonNew("deuxième valeur")
print(f"s2.valeur : {s2.valeur}")
print(f"id(s2) : {id(s2)}")

print(f"\ns1 is s2 : {s1 is s2}")

In [ ]:
# Méthode 2 : Métaclasse (plus avancé)
class SingletonMeta(type):
    """Métaclasse pour créer des Singletons"""
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            print(f"Création de l'instance {cls.__name__}")
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Configuration(metaclass=SingletonMeta):
    def __init__(self):
        self.settings = {}
    
    def set(self, key, value):
        self.settings[key] = value
    
    def get(self, key, default=None):
        return self.settings.get(key, default)

# Test
config1 = Configuration()
config1.set('database', 'postgresql')
print(f"config1.get('database') : {config1.get('database')}")

config2 = Configuration()
print(f"config2.get('database') : {config2.get('database')}")
print(f"config1 is config2 : {config1 is config2}")

In [ ]:
# Méthode 3 : Module (la plus pythonique)
# En Python, les modules sont des singletons naturels !

# Fichier : config.py (simulation)
class _ConfigurationModule:
    def __init__(self):
        self.settings = {}
    
    def set(self, key, value):
        self.settings[key] = value
    
    def get(self, key, default=None):
        return self.settings.get(key, default)

# Instance unique créée au chargement du module
config = _ConfigurationModule()

# Utilisation
config.set('host', 'localhost')
config.set('port', 5432)

print(f"Configuration : {config.settings}")
print("\nC'est l'approche recommandée en Python !")

In [ ]:
# Méthode 4 : Décorateur
def singleton(cls):
    """Décorateur pour transformer une classe en Singleton"""
    instances = {}
    
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    return get_instance

@singleton
class Logger:
    def __init__(self):
        self.logs = []
    
    def log(self, message):
        self.logs.append(message)
        print(f"[LOG] {message}")
    
    def get_logs(self):
        return self.logs

# Test
logger1 = Logger()
logger1.log("Premier message")

logger2 = Logger()
logger2.log("Deuxième message")

print(f"\nlogger1 is logger2 : {logger1 is logger2}")
print(f"Tous les logs : {logger1.get_logs()}")

## 3. Factory : Créer des Objets sans Spécifier la Classe

Le pattern **Factory** délègue la création d'objets à des méthodes ou classes spécialisées.

### Cas d'usage
- Créer différents types d'objets selon un paramètre
- Isoler la logique de création complexe
- Faciliter l'ajout de nouveaux types

In [ ]:
# Exemple : Factory pour différents types de documents
from abc import ABC, abstractmethod

class Document(ABC):
    """Interface commune pour tous les documents"""
    
    @abstractmethod
    def ouvrir(self):
        pass
    
    @abstractmethod
    def sauvegarder(self, contenu):
        pass

class DocumentPDF(Document):
    def ouvrir(self):
        return "Ouverture d'un document PDF"
    
    def sauvegarder(self, contenu):
        return f"Sauvegarde en PDF : {contenu}"

class DocumentWord(Document):
    def ouvrir(self):
        return "Ouverture d'un document Word"
    
    def sauvegarder(self, contenu):
        return f"Sauvegarde en DOCX : {contenu}"

class DocumentTexte(Document):
    def ouvrir(self):
        return "Ouverture d'un fichier texte"
    
    def sauvegarder(self, contenu):
        return f"Sauvegarde en TXT : {contenu}"

# Factory
class DocumentFactory:
    """Factory pour créer des documents"""
    
    _types = {
        'pdf': DocumentPDF,
        'word': DocumentWord,
        'txt': DocumentTexte,
    }
    
    @classmethod
    def creer_document(cls, type_doc):
        """Crée un document selon son type"""
        type_doc = type_doc.lower()
        if type_doc not in cls._types:
            raise ValueError(f"Type de document inconnu : {type_doc}")
        return cls._types[type_doc]()
    
    @classmethod
    def enregistrer_type(cls, nom, classe):
        """Permet d'ajouter dynamiquement de nouveaux types"""
        cls._types[nom] = classe

# Utilisation
for type_doc in ['pdf', 'word', 'txt']:
    doc = DocumentFactory.creer_document(type_doc)
    print(f"\n{type_doc.upper()} :")
    print(f"  {doc.ouvrir()}")
    print(f"  {doc.sauvegarder('Contenu du document')}")

# Test type inconnu
try:
    DocumentFactory.creer_document('excel')
except ValueError as e:
    print(f"\nErreur : {e}")

In [ ]:
# Factory pythonique avec dictionnaire et fonctions
def creer_pdf(titre, auteur):
    return {'type': 'PDF', 'titre': titre, 'auteur': auteur, 'pages': []}

def creer_word(titre, auteur):
    return {'type': 'DOCX', 'titre': titre, 'auteur': auteur, 'sections': []}

def creer_texte(titre):
    return {'type': 'TXT', 'titre': titre, 'lignes': []}

# Factory simple avec dictionnaire
DOCUMENT_FACTORY = {
    'pdf': creer_pdf,
    'word': creer_word,
    'txt': creer_texte,
}

def creer_document(type_doc, **kwargs):
    """Factory function pythonique"""
    factory = DOCUMENT_FACTORY.get(type_doc)
    if factory is None:
        raise ValueError(f"Type inconnu : {type_doc}")
    return factory(**kwargs)

# Utilisation
pdf = creer_document('pdf', titre="Mon PDF", auteur="Alice")
word = creer_document('word', titre="Mon Word", auteur="Bob")
txt = creer_document('txt', titre="Mon Texte")

print(f"PDF : {pdf}")
print(f"Word : {word}")
print(f"Texte : {txt}")

## 4. Strategy : Changer d'Algorithme à l'Exécution

Le pattern **Strategy** permet de définir une famille d'algorithmes, de les encapsuler et de les rendre interchangeables.

### Cas d'usage
- Différentes méthodes de tri
- Différentes stratégies de compression
- Différents modes de paiement
- Différents algorithmes de calcul

In [ ]:
# Approche orientée objet classique
from abc import ABC, abstractmethod

class StrategieCompression(ABC):
    """Interface pour les stratégies de compression"""
    
    @abstractmethod
    def compresser(self, donnees):
        pass

class CompressionZIP(StrategieCompression):
    def compresser(self, donnees):
        return f"[ZIP] Données compressées : {donnees[:20]}..."

class CompressionGZIP(StrategieCompression):
    def compresser(self, donnees):
        return f"[GZIP] Données compressées : {donnees[:20]}..."

class CompressionBZ2(StrategieCompression):
    def compresser(self, donnees):
        return f"[BZ2] Données compressées : {donnees[:20]}..."

class Compresseur:
    """Contexte qui utilise une stratégie"""
    
    def __init__(self, strategie: StrategieCompression):
        self.strategie = strategie
    
    def set_strategie(self, strategie: StrategieCompression):
        """Changer de stratégie à l'exécution"""
        self.strategie = strategie
    
    def compresser_fichier(self, donnees):
        return self.strategie.compresser(donnees)

# Utilisation
donnees = "Lorem ipsum dolor sit amet, consectetur adipiscing elit" * 10

compresseur = Compresseur(CompressionZIP())
print(compresseur.compresser_fichier(donnees))

# Changer de stratégie
compresseur.set_strategie(CompressionGZIP())
print(compresseur.compresser_fichier(donnees))

compresseur.set_strategie(CompressionBZ2())
print(compresseur.compresser_fichier(donnees))

In [ ]:
# Approche pythonique avec fonctions (plus simple)
def compression_zip(donnees):
    return f"[ZIP] Données compressées : {donnees[:20]}..."

def compression_gzip(donnees):
    return f"[GZIP] Données compressées : {donnees[:20]}..."

def compression_bz2(donnees):
    return f"[BZ2] Données compressées : {donnees[:20]}..."

class CompresseurPythonique:
    """Version pythonique avec callables"""
    
    def __init__(self, strategie_compression):
        self.strategie = strategie_compression
    
    def compresser_fichier(self, donnees):
        return self.strategie(donnees)

# Utilisation
donnees = "Lorem ipsum dolor sit amet" * 10

compresseur = CompresseurPythonique(compression_zip)
print(compresseur.compresser_fichier(donnees))

# Changer de stratégie
compresseur.strategie = compression_gzip
print(compresseur.compresser_fichier(donnees))

# Même avec une lambda
compresseur.strategie = lambda d: f"[CUSTOM] {d[:15]}..."
print(compresseur.compresser_fichier(donnees))

In [ ]:
# Exemple pratique : stratégies de tri
class TrieurProduits:
    """Trie des produits selon différentes stratégies"""
    
    def __init__(self, produits):
        self.produits = produits
    
    def trier(self, strategie):
        """Trie avec une stratégie donnée (callable)"""
        return sorted(self.produits, key=strategie)

# Produits
produits = [
    {'nom': 'Laptop', 'prix': 999, 'note': 4.5},
    {'nom': 'Souris', 'prix': 25, 'note': 4.2},
    {'nom': 'Clavier', 'prix': 75, 'note': 4.8},
    {'nom': 'Écran', 'prix': 350, 'note': 4.6},
]

trieur = TrieurProduits(produits)

# Différentes stratégies
print("Tri par prix croissant :")
for p in trieur.trier(lambda x: x['prix']):
    print(f"  {p['nom']:10} - {p['prix']:4}€ - Note: {p['note']}")

print("\nTri par note décroissante :")
for p in trieur.trier(lambda x: -x['note']):
    print(f"  {p['nom']:10} - {p['prix']:4}€ - Note: {p['note']}")

print("\nTri par nom :")
for p in trieur.trier(lambda x: x['nom']):
    print(f"  {p['nom']:10} - {p['prix']:4}€ - Note: {p['note']}")

## 5. Observer : Notification de Changements

Le pattern **Observer** définit une dépendance un-à-plusieurs entre objets : quand l'objet observé change, tous ses observateurs sont notifiés.

### Cas d'usage
- Système d'événements
- Interface utilisateur réactive
- Mise à jour de vues multiples
- Pub/Sub (publication/souscription)

In [ ]:
# Implémentation classique avec classes
class Sujet:
    """Objet observé (Subject)"""
    
    def __init__(self):
        self._observateurs = []
        self._etat = None
    
    def attacher(self, observateur):
        """Ajouter un observateur"""
        if observateur not in self._observateurs:
            self._observateurs.append(observateur)
    
    def detacher(self, observateur):
        """Retirer un observateur"""
        try:
            self._observateurs.remove(observateur)
        except ValueError:
            pass
    
    def notifier(self):
        """Notifier tous les observateurs"""
        for observateur in self._observateurs:
            observateur.mise_a_jour(self)
    
    @property
    def etat(self):
        return self._etat
    
    @etat.setter
    def etat(self, valeur):
        self._etat = valeur
        self.notifier()  # Notifier automatiquement

class Observateur(ABC):
    """Interface pour les observateurs"""
    
    @abstractmethod
    def mise_a_jour(self, sujet):
        pass

class ObservateurConcret(Observateur):
    def __init__(self, nom):
        self.nom = nom
    
    def mise_a_jour(self, sujet):
        print(f"[{self.nom}] Notification reçue ! Nouvel état : {sujet.etat}")

# Utilisation
sujet = Sujet()

obs1 = ObservateurConcret("Observateur 1")
obs2 = ObservateurConcret("Observateur 2")
obs3 = ObservateurConcret("Observateur 3")

# Attacher les observateurs
sujet.attacher(obs1)
sujet.attacher(obs2)
sujet.attacher(obs3)

# Changement d'état → notification automatique
print("Changement d'état à 'ACTIF' :")
sujet.etat = "ACTIF"

print("\nDétacher obs2")
sujet.detacher(obs2)

print("\nChangement d'état à 'INACTIF' :")
sujet.etat = "INACTIF"

In [ ]:
# Version pythonique avec callbacks
class SujetPythonique:
    """Sujet avec callbacks (plus pythonique)"""
    
    def __init__(self):
        self._callbacks = []
        self._etat = None
    
    def abonner(self, callback):
        """Ajouter un callback"""
        if callback not in self._callbacks:
            self._callbacks.append(callback)
    
    def desabonner(self, callback):
        """Retirer un callback"""
        try:
            self._callbacks.remove(callback)
        except ValueError:
            pass
    
    def notifier(self):
        """Appeler tous les callbacks"""
        for callback in self._callbacks:
            callback(self._etat)
    
    @property
    def etat(self):
        return self._etat
    
    @etat.setter
    def etat(self, valeur):
        self._etat = valeur
        self.notifier()

# Utilisation avec fonctions
def logger(etat):
    print(f"[LOGGER] État changé : {etat}")

def alerter(etat):
    if etat == "ERREUR":
        print(f"[ALERTE] État d'erreur détecté !")

def sauvegarder(etat):
    print(f"[SAUVEGARDE] État sauvegardé : {etat}")

# Test
sujet = SujetPythonique()
sujet.abonner(logger)
sujet.abonner(alerter)
sujet.abonner(sauvegarder)

print("Changement d'état à 'OK' :")
sujet.etat = "OK"

print("\nChangement d'état à 'ERREUR' :")
sujet.etat = "ERREUR"

# Même avec des lambdas
sujet.abonner(lambda e: print(f"[LAMBDA] Reçu : {e}"))
print("\nChangement d'état à 'INFO' :")
sujet.etat = "INFO"

In [ ]:
# Exemple pratique : système d'événements
class EventManager:
    """Gestionnaire d'événements générique"""
    
    def __init__(self):
        self._events = {}  # {event_name: [callbacks]}
    
    def on(self, event_name, callback):
        """Abonner à un événement"""
        if event_name not in self._events:
            self._events[event_name] = []
        self._events[event_name].append(callback)
    
    def off(self, event_name, callback):
        """Se désabonner d'un événement"""
        if event_name in self._events:
            try:
                self._events[event_name].remove(callback)
            except ValueError:
                pass
    
    def emit(self, event_name, *args, **kwargs):
        """Émettre un événement"""
        if event_name in self._events:
            for callback in self._events[event_name]:
                callback(*args, **kwargs)

# Utilisation
events = EventManager()

# Définir des handlers
def on_user_login(username):
    print(f"[LOGIN] Utilisateur connecté : {username}")

def on_user_logout(username):
    print(f"[LOGOUT] Utilisateur déconnecté : {username}")

def log_activity(action, username):
    print(f"[ACTIVITY] {username} -> {action}")

# S'abonner aux événements
events.on('login', on_user_login)
events.on('login', lambda u: log_activity('login', u))
events.on('logout', on_user_logout)
events.on('logout', lambda u: log_activity('logout', u))

# Émettre des événements
print("Événement : login")
events.emit('login', 'alice')

print("\nÉvénement : logout")
events.emit('logout', 'alice')

## 6. Decorator Pattern vs @decorator Python

**Attention** : Ne pas confondre !

- **Decorator Pattern** (GoF) : ajouter dynamiquement des responsabilités à un objet
- **@decorator Python** : fonction qui modifie une autre fonction

### Decorator Pattern (structure)

In [ ]:
# Decorator Pattern classique
class Composant(ABC):
    """Interface commune"""
    
    @abstractmethod
    def operation(self):
        pass
    
    @abstractmethod
    def cout(self):
        pass

class Cafe(Composant):
    """Composant de base"""
    
    def operation(self):
        return "Café"
    
    def cout(self):
        return 2.0

class Decorateur(Composant):
    """Décorateur de base"""
    
    def __init__(self, composant: Composant):
        self._composant = composant
    
    def operation(self):
        return self._composant.operation()
    
    def cout(self):
        return self._composant.cout()

class AvecLait(Decorateur):
    """Ajoute du lait"""
    
    def operation(self):
        return self._composant.operation() + " + Lait"
    
    def cout(self):
        return self._composant.cout() + 0.5

class AvecSucre(Decorateur):
    """Ajoute du sucre"""
    
    def operation(self):
        return self._composant.operation() + " + Sucre"
    
    def cout(self):
        return self._composant.cout() + 0.2

class AvecCreme(Decorateur):
    """Ajoute de la crème"""
    
    def operation(self):
        return self._composant.operation() + " + Crème"
    
    def cout(self):
        return self._composant.cout() + 0.7

# Utilisation : empilement de décorateurs
cafe = Cafe()
print(f"{cafe.operation()} : {cafe.cout()}€")

cafe_au_lait = AvecLait(cafe)
print(f"{cafe_au_lait.operation()} : {cafe_au_lait.cout()}€")

cafe_complet = AvecCreme(AvecSucre(AvecLait(cafe)))
print(f"{cafe_complet.operation()} : {cafe_complet.cout()}€")

In [ ]:
# @decorator Python (différent !)
def avec_log(func):
    """Décorateur qui ajoute du logging"""
    def wrapper(*args, **kwargs):
        print(f"[LOG] Appel de {func.__name__}")
        resultat = func(*args, **kwargs)
        print(f"[LOG] Fin de {func.__name__}")
        return resultat
    return wrapper

def avec_timer(func):
    """Décorateur qui mesure le temps"""
    import time
    def wrapper(*args, **kwargs):
        debut = time.time()
        resultat = func(*args, **kwargs)
        fin = time.time()
        print(f"[TIMER] {func.__name__} a pris {(fin-debut)*1000:.2f}ms")
        return resultat
    return wrapper

@avec_log
@avec_timer
def calculer(n):
    """Fonction décorée"""
    return sum(range(n))

# Utilisation
resultat = calculer(1000000)
print(f"Résultat : {resultat}")

## 7. Patterns Pythonic vs Classiques

Python a des idiomes qui simplifient ou remplacent certains patterns classiques.

In [ ]:
# Iterator Pattern : intégré en Python
class RangePairs:
    """Itérateur personnalisé"""
    
    def __init__(self, n):
        self.n = n
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        result = (self.i, self.i * 2)
        self.i += 1
        return result

# Ou plus pythonique avec un générateur
def range_pairs(n):
    """Générateur (plus pythonique)"""
    for i in range(n):
        yield (i, i * 2)

print("Avec classe Iterator :")
for a, b in RangePairs(5):
    print(f"  {a} -> {b}")

print("\nAvec générateur :")
for a, b in range_pairs(5):
    print(f"  {a} -> {b}")

In [ ]:
# Command Pattern : souvent remplacé par des callables
# Classique (verbeux)
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

class CommandOuvrir(Command):
    def execute(self):
        print("Ouvrir le fichier")

# Pythonique (simple)
def ouvrir():
    print("Ouvrir le fichier")

def sauvegarder():
    print("Sauvegarder le fichier")

def fermer():
    print("Fermer le fichier")

# Invoker
class MenuPythonique:
    def __init__(self):
        self.commandes = {}
    
    def ajouter_commande(self, nom, commande):
        self.commandes[nom] = commande
    
    def executer(self, nom):
        if nom in self.commandes:
            self.commandes[nom]()

menu = MenuPythonique()
menu.ajouter_commande('ouvrir', ouvrir)
menu.ajouter_commande('sauvegarder', sauvegarder)
menu.ajouter_commande('fermer', fermer)

menu.executer('ouvrir')
menu.executer('sauvegarder')

## Pièges Courants

### 1. Singleton et État Global

In [ ]:
# Problème : Singleton crée un état global difficile à tester
# Alternative : Dependency Injection

# Mauvais : Singleton global
class ConfigSingleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.settings = {}
        return cls._instance

# Bon : Dependency Injection
class Service:
    def __init__(self, config):
        self.config = config  # Injecté
    
    def faire_quelque_chose(self):
        return self.config.get('setting')

# Facile à tester
config_test = {'setting': 'valeur_test'}
service = Service(config_test)
print(f"Service avec config injectée : {service.faire_quelque_chose()}")

### 2. Over-Engineering

Ne pas utiliser un pattern si une solution simple suffit.

In [ ]:
# Over-engineering : Factory pour 2 types
class AnimalFactory:
    @staticmethod
    def creer_animal(type_animal):
        if type_animal == 'chien':
            return Chien()
        elif type_animal == 'chat':
            return Chat()

# Plus simple et suffisant
ANIMAUX = {
    'chien': Chien,
    'chat': Chat,
}

def creer_animal(type_animal):
    return ANIMAUX[type_animal]()

print("Préférez la simplicité !")

### 3. Confondre Decorator Pattern et @decorator

In [ ]:
# Ce ne sont PAS la même chose !
print("Decorator Pattern : wrapper d'objets pour ajouter des fonctionnalités")
print("@decorator Python : modification de fonctions au moment de la définition")
print("\nLes deux ajoutent des fonctionnalités, mais de manière très différente")

## Mini-Exercices

### Exercice 1 : Singleton pour Configuration

In [ ]:
# Créez un Singleton de configuration avec :
# - Chargement depuis un fichier (simulé)
# - get(key, default)
# - set(key, value)
# - reload() pour recharger

# Votre code ici

### Exercice 2 : Factory pour Parsers

In [ ]:
# Créez une Factory pour différents parsers :
# - JSONParser
# - XMLParser  
# - CSVParser
# Chacun avec une méthode parse(data)

# Votre code ici

### Exercice 3 : Observer Simple

In [ ]:
# Créez un système Observer pour un stock de produits :
# - Stock(produit, quantite)
# - Observateurs : EmailNotifier, SMSNotifier, Logger
# - Notification quand stock < seuil

# Votre code ici

## Conclusion

Les design patterns sont des outils puissants, mais doivent être utilisés avec discernement.

### Points clés

1. **Singleton** : une seule instance (mais attention à l'état global)
2. **Factory** : isoler la création d'objets
3. **Strategy** : algorithmes interchangeables (très pythonique avec callables)
4. **Observer** : notification de changements (pub/sub)
5. **Decorator Pattern** ≠ @decorator Python

### Approche pythonique

- Préférer la simplicité aux patterns complexes
- Utiliser les fonctions de première classe
- Les modules sont des singletons naturels
- Les générateurs remplacent Iterator
- Les callables simplifient Strategy et Command

### Quand utiliser un pattern ?

- Le problème correspond exactement au pattern
- La solution apporte plus de clarté que de complexité
- Le code sera maintenu et évolué
- L'équipe connaît le vocabulaire des patterns